# The Cost of Virtual Method Calls

Virtual methods are widely thought to be slower than non-virtual methods.  Let's explore whether and why that might be.

In [1]:
from cfiddle import *
from cfiddle.jupyter import *


Here's a simple test that invokes the same function statically and dynamically.  We'll compile it with basic optimizations but not inlining so we can see the cost of the function call itself.

In [15]:
static_vs_virtual = code(r"""
#include"cfiddle.hpp"

class A {
public:
    virtual int foo(int x) {
        int s = 0;
        for(int i = 0; i < 10; i++) {
            s += x;
        }
        return s;
    }
};

extern "C"
int static_call(uint64_t iterations) {
	A a;
	register int sum = 0;
	
    start_measurement();
	for(register uint64_t i = 0; i < iterations; i++)
		sum += a.foo(4);
    end_measurement();

	return sum;
}


extern "C"
int virtual_call(uint64_t iterations) {
	register A * a = new A();
	register int sum = 0;
	
    start_measurement();
	for(register uint64_t i = 0; i < iterations; i++)
		sum += a->foo(4);
	end_measurement();
	return sum;
}

""")

## The Cost of Calling A Function

In [16]:
exe = build(static_vs_virtual, build_parameters=dict(OPTIMIZE="-Og -fno-inline"))
display(compare([exe[0].cfg("static_call"), exe[0].cfg("virtual_call")], ["static_call", "virtual_call"]))
results = run(executable=exe, function=["static_call", "virtual_call"], arguments=[dict(iterations=100000000)])
display(results.as_df())

,OPTIMIZE,function,iterations,ET
0,-Og -fno-inline,static_call,100000000,0.570782
1,-Og -fno-inline,virtual_call,100000000,0.541129


There's not much difference:  The virtual invocation uses one more instruction, but the performance difference is in the noise.

## With Inlining

Now, we'll turn on `-O3` and see what happens.

In [17]:
exe = build(static_vs_virtual, build_parameters=dict(OPTIMIZE="-O3"))
display(compare([exe[0].cfg("static_call"), exe[0].cfg("virtual_call")], ["static_call -O3", "virtual_call -O3"]))
results = run(executable=exe, function=["static_call", "virtual_call"], arguments=arg_map(iterations=[100000000]))
display(results.as_df())

,OPTIMIZE,function,iterations,ET
0,-O3,static_call,100000000,0.000008
1,-O3,virtual_call,100000000,0.056824


Wow!  The static call is inlined away -- there's nothing left and the execution time is constant.  The virtual invocation is faster with `-O3` (and to be honest, I'm not sure what the compiler did), but execution time is still linear in `iterations` (Try it and see!). 

Now we can see the real cost of virtual function calls:  It's not that virtual function calls are expensive, it's that they the cripple the compiler's ability to inline and perform the attendant optimizations (in this case constant propagation and static evaluation).